## Individual Project
   He Man 01387756

   [1. Data Preparation](#1)<br>
   [2. Table1: Descriptive Statistics (BDI included)](#2)<br>
   [3. Table2: Unit Root Test (BDI included)](#3)<br>
   [4. * **Table3**: Linear Regression of Iron Ore Prices on Explanatory Variable based on Eq.(3)](#4)<br>
   [5. * **Table4**: Regression of Freight Prices on Explanatory Variables based on Eq.(6)](#5)<br>
   [6. * **Table5**: 2SLS Panel Regression of Freight Prices on Explanatory Variables based on Eq.(7)](#6)<br>
   [7. Table6: Tests on Estimated Residuals based on Eq.(7)](#7)<br>
   
Other:
[Conclusion (About Endogeneity)](#8)<br>

In [1]:
import numpy as np
import pandas as pd
import statsmodels.formula.api as smf
from statsmodels.tsa.stattools import adfuller
import statsmodels.api as sm
from linearmodels.panel import PanelOLS
import scipy.stats as ss

<a id='1'></a>
### Data Preparation
   - JC1: data to do regression based on Eq.(3)
   - JC3: data to do regression based on Eqs.(6) and (7)

In [2]:
JC1 = pd.read_csv('JC1.csv', header=0).drop(columns=['t','HSFO', 'prod', 'difprod', \
                                                    'difiron', 'season1', 'season2', 'season3', \
                                                    'season4', 'lnf'])
JC1.head(2)

,yyyymm,ore_price,growth,VIX,BDI
0,201312,109.73000,0.005,13.72,2277
1,201401,110.38838,0.006,18.41,1110


In [3]:
JC3 = pd.read_csv('amended_JC3.csv', header=0).drop(columns=['year', 'dist', 'HSFO', 'points'])
JC3.head(2)

,date,port,ore_price,growth,VIX,logd,logf,avefreight,aveqty,BDI
0,201312,DAMPIER,109.73000,0.005,13.72,8.183677,6.388561,13.367059,210000.0000,2277
1,201401,DAMPIER,110.38838,0.006,18.41,8.183677,6.380123,7.676923,271538.4615,1110


<a id='2'></a>
### Table 1: Descriptive Statistics
   Descriptive Statistics of Variables Used in the Study: iron-ore price in USD per metric ton, PRC industrial production growth rate, VIX index, natural log of distance in nautical miles, natural log of high Sulphur 380 bunker fuel, freight rates in USD per metric ton of cargo, cargo volumes in metric tons, and BDI index. The sample period is from December 2013 to May 2019. Sample size is 264. The mean, standard deviation, 25th percentile (25%), median, and 75th percentile (75%) of the various time series are reported.

In [4]:
table1 = JC3.copy()
table1['Cargo_Volume*10^4'] = table1['aveqty']/10000
table1['Growth_Rate*10^-3'] = table1['growth']*1000
table1['BDI*10^2'] = table1['BDI']/100
table1[['ore_price', 'Growth_Rate*10^-3', 'VIX', 'logd', 'logf', 'avefreight', 'Cargo_Volume*10^4', \
        'BDI*10^2']].describe().T.drop(columns=['count','min','max'])

,mean,std,25%,50%,75%
ore_price,72.279830,18.004146,59.090000,68.415000,81.060000
Growth_Rate*10^-3,5.393939,1.447471,5.000000,5.000000,6.000000
VIX,15.075606,3.799299,12.370000,13.975000,16.950001
logd,8.666316,0.497286,8.183886,8.584074,9.066504
logf,5.946294,0.301473,5.802118,5.964725,6.173786
avefreight,9.619252,4.887627,5.913348,8.178897,12.319167
Cargo_Volume*10^4,20.819793,5.405522,17.000000,18.390152,22.962500
BDI*10^2,9.971818,3.670256,7.030000,9.445000,12.310000


<a id='3'></a>
### Table 2: Unit Root Test of Iron Ore price, Growth rate, VIX index and BDI index time series over monthly data.
   Similar as paper, here we also do unit root test on the new added BDI index.

In [5]:
table2 = pd.DataFrame(columns=['ore_price', 'growth', 'VIX', 'BDI'], \
                   index=['ADF-Statistics','p-Value','1%','5%','10%'])

for i in range(4):
    result = adfuller(JC1.iloc[:,i+1])
    table2.iloc[0,i] = format(result[0],'.4f')
    table2.iloc[1,i] = format(result[1],'.3f')
    CV = list(result[4].values()) #critical values
    table2.iloc[2,i] = format(CV[0],'.3f')
    table2.iloc[3,i] = format(CV[1],'.3f')
    table2.iloc[4,i] = format(CV[2],'.3f')

table2

,ore_price,growth,VIX,BDI
ADF-Statistics,-2.0857,-9.3395,-5.0541,-4.3414
p-Value,0.250,0.000,0.000,0.000
1%,-3.539,-3.535,-3.535,-3.535
5%,-2.909,-2.907,-2.907,-2.907
10%,-2.592,-2.591,-2.591,-2.591


#### Conclusion
From the last column of Table2, it is seen that BDI index is also stationary or I(0) process.

<a id='4'></a>
### Table 3: Linear Regression of Iron Ore Prices on Explanatory Variable based on Eq.(3).

1. Linear regression based on Eq.(3) using White(1980)'s heteroscedastic consistent covariance matrix estimator (**HCCME**).
2. **ADF unit root test** to check for co-integration.
<br>

Adjusted Equation(3):
$$P^{R}_{t}=\theta_{0}+\theta_{1}X_{t}-\theta_{2}VIX_{t}+\theta_{3}BDI_{t}+\epsilon_{t}$$

We assume this zero mean residual error is independent across time, but can be heteroskedastic.

In [6]:
table3 = JC1.copy()
table3['n_VIX'] = table3['VIX']*(-1)
table3.head(3)

,yyyymm,ore_price,growth,VIX,BDI,n_VIX
0,201312,109.73000,0.005,13.72,2277,-13.72
1,201401,110.38838,0.006,18.41,1110,-18.41
2,201402,111.05234,0.006,14.00,1258,-14.00


In [7]:
table3_res = smf.ols('ore_price ~ 1 + growth + n_VIX + BDI', data = table3).fit(cov_type = 'HC1')
table3_res.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              ore_price   R-squared:                       0.267
Model:                            OLS   Adj. R-squared:                  0.232
Method:                 Least Squares   F-statistic:                     9.084
Date:                Mon, 27 Apr 2020   Prob (F-statistic):           4.52e-05
Time:                        00:11:09   Log-Likelihood:                -274.04
No. Observations:                  66   AIC:                             556.1
Df Residuals:                      62   BIC:                             564.8
Df Model:                           3                                         
Covariance Type:                  HC1                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     39.0977     12.379      3.158      0.002      14.835      63.360
growth      3754.9578   1527.034      2.459      0.014     762.027    6747.889
n_VIX          0.4170      0.410      1.018      0.309      -0.386       1.220
BDI            0.0193      0.005      4.012      0.000       0.010       0.029
==============================================================================
Omnibus:                        9.960   Durbin-Watson:                   0.487
Prob(Omnibus):                  0.007   Jarque-Bera (JB):               10.314
Skew:                           0.962   Prob(JB):                      0.00576
Kurtosis:                       3.222   Cond. No.                     7.37e+05
==============================================================================

Warnings:
[1] Standard Errors are heteroscedasticity robust (HC1)
[2] The condition number is large, 7.37e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [8]:
table3_adf = adfuller(table3_res.resid)
print('ADF-statistics Residual Error: {}, p-Value: {}'.format(round(table3_adf[0],3),round(table3_adf[1],3)))

ADF-statistics Residual Error: -2.672, p-Value: 0.079


#### Conclusion
1. Still, the **growth** of China's production index is **significant** in pushing up iron ore prices as seen in the positive coef at p-value of 0.014(<0.05).
2. **VIX** is **not significant** as we can not reject the null hypothesis with p-value of 0.309.
3. Aso, the new factor **BDI** is **significant** too with positive coef at p-value of 0.000(<0.01).
4. ADF statisitc on estimated residuals $\hat{\epsilon_{t}}$ shows **unit root is rejected** at a p value of 0.079(<0.1), so the regression is co-integrated.

<a id='5'></a>
### Table4: Regression of Freight Prices on Explanatory Variables based on Eq.(6).

1. Panel regression with fixed port treatment effect and clustered standard errors.

Adjusted Equation(6):
$$P^{S}_{jt}=\gamma_{0}+\gamma_{1}X_{t}+\gamma_{2}D_{j}+\gamma_{3}B_{t}+\gamma_{4}P^{R}_{t}+\gamma_{5}BDI_{t}+\eta_{jt}$$

Both the distance and the fuel price are transformed by taking natural logarithms.

In [9]:
table4 = JC3.copy()
table4['n_ore_price'] = table4['ore_price']*(-1)
table4 = table4.set_index(['port','date'])
table4.head(3)

ore_price  growth    VIX      logd      logf  avefreight  \
port    date                                                               
DAMPIER 201312  109.73000   0.005  13.72  8.183677  6.388561   13.367059   
        201401  110.38838   0.006  18.41  8.183677  6.380123    7.676923   
        201402  111.05234   0.006  14.00  8.183677  6.375025    8.202500   

                     aveqty   BDI  n_ore_price  
port    date                                    
DAMPIER 201312  210000.0000  2277   -109.73000  
        201401  271538.4615  1110   -110.38838  
        201402  260833.3333  1258   -111.05234

In [10]:
exog_vars = ["growth", "logf", "ore_price", "BDI"] #as we use fixed entity effect here, we drop logd(distance)
exog = sm.add_constant(table4[exog_vars])
mod = PanelOLS(table4.avefreight, exog, entity_effects = True) #entity_effects = True, we set port as fixed entity effect
table4_res = mod.fit(cov_type = 'clustered', cluster_entity = True)
table4_res

Dep. Variable:,avefreight,R-squared:,0.6707
Estimator:,PanelOLS,R-squared (Between):,0.0000
No. Observations:,264,R-squared (Within):,0.6707
Date:,"Mon, Apr 27 2020",R-squared (Overall):,0.3376
Time:,00:11:09,Log-likelihood,-555.74
Cov. Estimator:,Clustered,,
,,F-statistic:,130.36
Entities:,4,P-value,0.0000
Avg Obs:,66.000,Distribution:,"F(4,256)"
Min Obs:,66.000,,
Max Obs:,66.000,F-statistic (robust):,1.189e+14


**From Table4:**
1. the growth of Chinese industrial production does have positive impact on the freight prices, however, this impact is not significant (p-value = 0.2326), unlike that on iron ore prices (p-value = 0.014).
2. Increases in bunker fuel oil increases freight rates significantly (p-value = 0.0056).
3. Iron ore price has positive impact on the freight prices but this impact is also not significant and almost negligible (p-value = 0.5723).
4. BDI index has significant positive impact on the freight prices (p-value = 0.000)


<a id='8'></a>
#### Conclusion (About Endogeneity)
We treat BDI index as a latent (but observable) variable affecting market iron ore prices and the iron bulk carrier shipping rates simultaneously and introduce it as an additional regressor in both Eqs.(3) and (6) to see if it plays the same role like the growth variable.<br>

From the result we can see, BDI index is important as it does have impact on both ore price and freight rate, but there is still presence of latent variables thus lead to existed endogeneity in iron ore prices in the regression of Eq.(6), as Table4 shows that the p-value of ore_price is 0.5723.

I also tried Pooled OLS and Panel OLS without fixed entity effect, and the result did not suprise me.

<a id='6'></a>
### Table5: 2SLS Panel Regression of Freight Prices on Explanatory Variables based on Eq.(7).
- First Stage: Obtain the time series of the estimated iron ore prices by using Eq.(3) regression. (we already did in table3 part) [Table3](#4)<br>
- Second Stage: Re-run the panel regression using instead the Eq.(7) as below.

Adjusted Equation(7):
$$P^{S}_{jt}=\gamma_{0}+\gamma_{1}X_{t}+\gamma_{2}D_{j}+\gamma_{3}B_{t}+\gamma_{4}\hat{P}^{R}_{t}+\gamma_{5}BDI_{t}+\nu_{jt}$$

In [11]:
table5_predict = table3[['yyyymm']].copy()
table5_predict['ore_predict'] = table3_res.predict()
table5_predict = table5_predict.rename(columns={'yyyymm': 'date'})

table5 = JC3.copy()
table5['index'] = table5.index.values
table5 = pd.merge(table5, table5_predict, on = 'date').sort_values(by = 'index')
table5.head(3)

,date,port,ore_price,growth,VIX,logd,logf,avefreight,aveqty,BDI,index,ore_predict
0,201312,DAMPIER,109.73000,0.005,13.72,8.183677,6.388561,13.367059,210000.0000,2277,0,96.026714
4,201401,DAMPIER,110.38838,0.006,18.41,8.183677,6.380123,7.676923,271538.4615,1110,1,75.338993
8,201402,DAMPIER,111.05234,0.006,14.00,8.183677,6.375025,8.202500,260833.3333,1258,2,80.029801


In [12]:
table5_1 = table5.copy().set_index(['port','date'])
exog_vars = ["growth", "logf", "ore_predict", "BDI"]
exog = sm.add_constant(table5_1[exog_vars])
mod = PanelOLS(table5_1.avefreight, exog, entity_effects = True)
table5_1_res = mod.fit(cov_type = 'clustered', cluster_entity = True)
table5_1_res

Dep. Variable:,avefreight,R-squared:,0.6727
Estimator:,PanelOLS,R-squared (Between):,1.11e-16
No. Observations:,264,R-squared (Within):,0.6727
Date:,"Mon, Apr 27 2020",R-squared (Overall):,0.3386
Time:,00:11:09,Log-likelihood,-554.96
Cov. Estimator:,Clustered,,
,,F-statistic:,131.52
Entities:,4,P-value,0.0000
Avg Obs:,66.000,Distribution:,"F(4,256)"
Min Obs:,66.000,,
Max Obs:,66.000,F-statistic (robust):,1.405e+13


#### Conclusion
1. Each dollar increase in iron ore per dry metric ton would correspond to a decrease ship freight cost by an average of 11.02 cents per metric ton, ceteris paribus.
2. Each 1% increase in China's industrial production growth rate would correspond to an increase of \$5.65 freight rate per ton, ceteris paribus.
3. Compared with [Table4](#5), the estimated coefficients for growth and expected iron price are now significant at 5% and 1% levels respectively. The BDI index is still significant but negligible at 1% level.

<a id='7'></a>
### Table6: Tests on Estimated Residuals based on Eq.(7).

In [13]:
table6 = pd.DataFrame(columns=['Estimate', 'p-Value'], \
                      index=['ADF-nu1', 'ADF-nu2','ADF-nu3','ADF-nu4','corr(ep,nu1)', \
                             'corr(ep,nu2)', 'corr(ep,nu3)','corr(ep,nu4)'])

table5_1_resids = table5_1_res.resids
ports = table5_1_resids.index.get_level_values(level = 'port').unique()
table5_1_resids_groups = table5_1_resids.groupby(level = 'port')
ep = table3_res.resid # estimated residuals epsilon from Eq.(3)
for i in range(4):
    table5_1_resids_group = table5_1_resids_groups.get_group(ports[i])
    res1 = adfuller(table5_1_resids_group)
    table6.iloc[i,0] = format(res1[0],'.3f')
    table6.iloc[i,1] = format(res1[1],'.3f')
    res2 = ss.pearsonr(ep, table5_1_resids_group)
    table6.iloc[i+4,0] = format(res2[0],'.3f')
    table6.iloc[i+4,1] = format(res2[1],'.3f')
table6

,Estimate,p-Value
ADF-nu1,-2.185,0.212
ADF-nu2,-2.293,0.174
ADF-nu3,-3.479,0.009
ADF-nu4,-3.178,0.021
"corr(ep,nu1)",-0.234,0.058
"corr(ep,nu2)",-0.197,0.114
"corr(ep,nu3)",0.310,0.011
"corr(ep,nu4)",0.108,0.387


**Conclusion**<br>
 Though the ADF test part is not satisfying, the correlation part shows that there might be still a common "undiscovered" element in the errors of epsilon and nu in port2 and port4, which is also an evidence of endogeneity of iron ore prices.